# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

1022709

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

arthur


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

1080

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['une_colonne', 'dee', 'nordiste', 'a', 'perce', 'les', 'lignes', 'jie', 'defense', 'du', 'han', 'l', "'", 'aerodrome', 'de', 'won', 'est', 'menace', 'les', 'bombardiers_americains', 'reagissent', 'energiquement', 'washington', ',', '30_juin', '(', 'a', ',', 'pl', 'les', 'blindes', 'de', 'la', 'coree', 'du', 'ifyrd', 'ont', 'perce', 'la', 'ligne', 'de', 'defense', 'sudiste', 'etablie', 'sur', 'le', 'fledve', 'ha', '>>;', 'et', 'ont', 'ainsi', 'dispose', 'de', 'la', 'principale', 'resistance', 'a', 'leur_avance', 'vers', 'le', 'sud', '.'], ['j', 'les', 'forces', 'de', 'la', 'coree', 'du', 'sud', ',', 'deja', 'vivement', 'eprouvees', ',', 'et', 'qui', 'avaient_recu', 'l', "'", 'ordre', 'de', 'mac_arthur', 'de', 'tenir', 'la', 'ligne', 'du', 'han', 'a', 'tout', 'prix', ',', 'nont', 'pu', 'resister', 'a', 'un_groupe', 'ce', '40', 'a', '50', 'tanks', 'et', 'vehicules_blindes', 'qui', 'ont_ete_lances', 'dans', 'la', 'bataille', 'par', 'les', 'communistes', ',', 'immediatement', 'au_sud', 'd

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 16.3 s, sys: 177 ms, total: 16.5 s
Wall time: 6.2 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [17]:
model.wv["guerre"]

array([-2.73394   ,  1.1337913 ,  0.40669492,  1.9797053 , -4.7787576 ,
        0.3724211 ,  0.08412026,  2.563389  , -1.2853564 ,  1.8219448 ,
        0.972109  , -6.2818866 , -2.7329767 ,  0.3063019 ,  2.1111825 ,
       -2.520036  , -0.55125207, -0.17013268, -0.51343006,  0.7840533 ,
        3.838268  , -0.890201  ,  0.29639888,  4.3204856 ,  1.7620552 ,
       -2.1350093 ,  4.744159  , -4.788518  ,  1.9212629 ,  0.45041132,
       -0.43642777, -0.19517875], dtype=float32)

### Calculer la similarité entre deux termes

In [18]:
model.wv.similarity("ministre", "roi")

0.7126774

### Chercher les mots les plus proches d'un terme donné

In [19]:
model.wv.most_similar("guerre", topn=10)

[('coree', 0.8069037795066833),
 ('chine', 0.8015821576118469),
 ('campagne', 0.7715840339660645),
 ('les_evenements', 0.7703481912612915),
 ('lutte', 0.7339529395103455),
 ('mandchourie', 0.7277742028236389),
 ('toutes_les_troupes_etrangeres', 0.7200883626937866),
 ('ligne', 0.7161356210708618),
 ('russie', 0.7106516361236572),
 ('liberation', 0.7013394832611084)]

In [20]:
model.wv.most_similar("gouvernement", topn=10)

[('gouvernement_britannique', 0.8833344578742981),
 ('gouvernement_sovietique', 0.822905957698822),
 ('gouvernement_americain', 0.8190860748291016),
 ('chancelier', 0.8177461624145508),
 ('kremlin', 0.7836843729019165),
 ('conseil', 0.763762354850769),
 ('parlement', 0.7635948061943054),
 ('gouvernement_francais', 0.7597137093544006),
 ('president_eisenhower', 0.7536940574645996),
 ('premier_ministre', 0.7530848979949951)]

In [21]:
model.wv.most_similar("coree", topn=10)

[('en_coree', 0.8746688365936279),
 ('chine', 0.8735305070877075),
 ('formose', 0.8349776864051819),
 ('guerre', 0.8069037795066833),
 ('mandchourie', 0.7714316248893738),
 ('chine_communiste', 0.7629627585411072),
 ('sud', 0.7339551448822021),
 ('russie', 0.7313560247421265),
 ('des_nations_-_unies', 0.7273073792457581),
 ('nord', 0.7196094393730164)]

In [22]:
model.wv.most_similar("general", topn=10)

[('chef', 0.8671518564224243),
 ('president', 0.8091784715652466),
 ('premier_ministre', 0.7929507493972778),
 ('marechal', 0.7752862572669983),
 ('communique', 0.7669019103050232),
 ('ministre', 0.7568894624710083),
 ('cabinet', 0.7533312439918518),
 ('colonel', 0.750442624092102),
 ('general_eisenhower', 0.7473904490470886),
 ('secretaire', 0.7458898425102234)]

In [23]:
model.wv.most_similar("politique", topn=5)

[('conclusion', 0.8591688275337219),
 ('menace', 0.8437749743461609),
 ('volonte', 0.8413587808609009),
 ('paix', 0.8402823805809021),
 ('position', 0.8400108814239502)]

In [24]:
model.wv.most_similar("president", topn=10)

[('president_du_conseil', 0.9065456986427307),
 ('ministre', 0.8825893998146057),
 ('premier_ministre', 0.8774160742759705),
 ('au_nom', 0.8607130646705627),
 ('delegue', 0.8532878756523132),
 ('secretaire', 0.8513514995574951),
 ('senateur', 0.8492695689201355),
 ('cabinet', 0.8483782410621643),
 ('conseiller', 0.8406466841697693),
 ('ministre_des_finances', 0.8119514584541321)]

In [25]:
model.wv.most_similar("pays", topn=10)

[('cas', 0.7969898581504822),
 ('probleme', 0.7299181818962097),
 ('territoire', 0.712498128414154),
 ('progres', 0.7116485238075256),
 ('moyen_orient', 0.7112061381340027),
 ('japon', 0.7008605003356934),
 ('mouvement', 0.6820946335792542),
 ('regime', 0.6699736714363098),
 ('conflit', 0.6693381071090698),
 ('developpement', 0.6653424501419067)]

In [26]:
model.wv.most_similar("ministre", topn=5)

[('premier_ministre', 0.8966644406318665),
 ('president', 0.8825893998146057),
 ('president_du_conseil', 0.8374601602554321),
 ('secretaire', 0.8367466330528259),
 ('ministere', 0.8325507044792175)]

In [27]:
model.wv.most_similar("forces", topn=6)

[('troupes', 0.8973504900932312),
 ('armees', 0.8633828163146973),
 ('allies', 0.8591504096984863),
 ('forces_americaines', 0.8407933115959167),
 ('operations', 0.838506281375885),
 ('frontieres', 0.8179177641868591)]

In [28]:
model.wv.most_similar("communiste", topn=7)

[('sovietique', 0.9028414487838745),
 ('britannique', 0.8819445967674255),
 ('russe', 0.8708498477935791),
 ('du_parti', 0.813569962978363),
 ('des_nations_-_unies', 0.7826350331306458),
 ('formose', 0.757843017578125),
 ('militaire', 0.7576049566268921)]

In [29]:
model.wv.most_similar("sovietique", topn=5)

[('britannique', 0.9223195314407349),
 ('russe', 0.9081735610961914),
 ('communiste', 0.9028413891792297),
 ('diplomatique', 0.8345897793769836),
 ('militaire', 0.8283159136772156)]

### Faire des recherches complexes à travers l'espace vectoriel

In [30]:
model.wv.most_similar("gouvernement", topn=5)

[('gouvernement_britannique', 0.8833344578742981),
 ('gouvernement_sovietique', 0.822905957698822),
 ('gouvernement_americain', 0.8190860748291016),
 ('chancelier', 0.8177461624145508),
 ('kremlin', 0.7836843729019165)]